In [20]:
#-*- coding: utf-8 -*-

# function to split syllables to characters
def split_syllables(word):
	top_list = [u'ㄱ', u'ㄲ', u'ㄴ', u'ㄷ', u'ㄸ', u'ㄹ', u'ㅁ', u'ㅂ', u'ㅃ', u'ㅅ', u'ㅆ', u'ㅇ', u'ㅈ', u'ㅉ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ']
	mid_list = [u'ㅏ', u'ㅐ', u'ㅑ', u'ㅒ', u'ㅓ', u'ㅔ', u'ㅕ', u'ㅖ', u'ㅗ', u'ㅘ', u'ㅙ', u'ㅚ', u'ㅛ', u'ㅜ', u'ㅝ', u'ㅞ', u'ㅟ', u'ㅠ', u'ㅡ', u'ㅢ', u'ㅣ']
	bot_list = [u'', u'ㄱ', u'ㄲ', u'ㄳ', u'ㄴ', u'ㄵ', u'ㄶ', u'ㄷ', u'ㄹ', u'ㄺ', u'ㄻ', u'ㄼ', u'ㄽ', u'ㄾ', u'ㄿ', u'ㅀ', u'ㅁ', u'ㅂ', u'ㅄ', u'ㅅ', u'ㅆ', u'ㅇ', u'ㅈ', u'ㅊ', u'ㅋ', u'ㅌ', u'ㅍ', u'ㅎ']
	result = u""
	for syllable in word:
		if ord(syllable) < 44032 or ord(syllable) > 55203:
			result += syllable
		else:
			base = ord(syllable)-44032
			top_idx = base//588
			mid_idx = (base%588)//28
			bot_idx = (base%588)%28
			result += top_list[top_idx] + mid_list[mid_idx] + bot_list[bot_idx]
	return result

# function to build syllable by joining characters
def combine_to_syllables(chars):
	top_idx = [0, 1, -1, 2, -1, -1, 3, 4, 5, -1, -1, -1, -1, -1, -1, -1, 6, 7, 8, -1, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
	bot_idx = [1, 2, 3, 4, 5, 6, 7, -1, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, -1, 18, 19, 20, 21, 22, -1, 23, 24, 25, 26, 27]
	idx = 0
	types = [0]*len(chars)
	for char in chars:
		if ord(char) > 12643 or ord(char) < 12593:
			types[idx] = -1
		elif ord(char) >= 12623:
			types[idx] = 2
		idx+=1
	for i in range(1, len(types)):
		if types[i] == 2 and types[i-1]==0:
			types[i-1]=1
	for i in range(0, len(types)-1):
		if types[i] == 2 and types[i+1]==0:
			types[i+1]=3
	syllables = []
	character = []
	idx = 0
	for i in types:
		if i<2 and len(character)>0:
			syllables.append(character)
			character = []
		character.append(chars[idx])
		if i==-1:
			syllables.append(character)
			character = []
		idx+=1
	if len(character)>0:
		syllables.append(character)
	result = u""
	for syllable in syllables:
		if len(syllable)==3:
			tmp = 44032 + top_idx[ord(syllable[0])-12593]*588 + (ord(syllable[1])-12623)*28 + bot_idx[ord(syllable[2])-12593]
			result += chr(tmp)
		elif len(syllable)==2:
			if ord(syllable[0])>=12623:
				result += syllable[0] + syllable[1]
				continue
			tmp = 44032 + top_idx[ord(syllable[0])-12593]*588 + (ord(syllable[1])-12623)*28
			result += chr(tmp)
		elif len(syllable)==1:
			result += syllable[0]
		else:
			for character in syllable:
				result+=character	
	return result

# function to print list
def print_list(arg):
	for i in range(len(arg)):
		print(arg[i])
	print("")

# function to print dictionary
def print_dict(arg):
	for key in arg.keys():
		print (key, ":", arg[key])
	print("")

In [21]:
# step1: load requried resources

# load dictionary
dictionary={}
with open("dictionary", "r") as fin:
	while True:
		line=fin.readline()
		if not line:	break
		line=line.strip()
		morph_tag= line.split('\t')
		if morph_tag[0] in dictionary:
			l= dictionary[morph_tag[0]]+[morph_tag[1]]
			dictionary[morph_tag[0]]=l
		else:
			dictionary[morph_tag[0]]=[morph_tag[1]]

print('dictionary:')
print_dict(dictionary)

# load connectivity
connectivity=[]
with open("connectivity", "r") as fin:
	while True:
		line=fin.readline()
		if not line:	break
		line=line.strip()
		connectivity+=[line]

print('connectivity:')
print_list(connectivity)


dictionary:
ㅁ : ['명사형어미']
가 : ['동사']
감 : ['명사', '동사']
감기 : ['명사', '동사']
기 : ['명사형어미']
는 : ['보조사', '관형사형어미']
먹 : ['명사', '동사']
은 : ['보조사', '관형사형어미']

connectivity:
명사	보조사
동사	명사형어미
동사	관형사형어미
명사형어미	보조사



In [22]:
# step2: split word into character
# read user input & syllables->characters
word= input("Input word: ")
chars= split_syllables(word)
print ("word: ", word)
print ("chars: ", chars)


Input word: 감기는
word:  감기는
chars:  ㄱㅏㅁㄱㅣㄴㅡㄴ


In [23]:
# step3: build CYK table
# construct empty table with word length
table=[[] for _ in range(len(chars))]
for i in range(len(chars)):
	table[i]=[[] for _ in range(i+1)]
print ("cyk table initialized:")
print_list(table)


cyk table initialized:
[[]]
[[], []]
[[], [], []]
[[], [], [], []]
[[], [], [], [], []]
[[], [], [], [], [], []]
[[], [], [], [], [], [], []]
[[], [], [], [], [], [], [], []]



In [24]:
# step4: morpheme segmentation

for idx_col in range(len(chars)-1, -1, -1):	# backward loop (len-1, len-2, ..., 0)
	for idx_row in range(len(chars)-1, idx_col-1, -1):
		# make pseudo syllables by combining the characters
		key = combine_to_syllables(chars[idx_col:idx_col+(len(chars)-idx_row)])
		# search dictionary using the pseudo syllables as the key
		if(key in dictionary.keys()):
			for value in dictionary[key]:
				table[idx_row][idx_col]+= [key+":"+value]

print ("cyk table after morpheme segmentation:")
print_list(table)


cyk table after morpheme segmentation:
[[]]
[[], []]
[[], [], []]
[['감기:명사', '감기:동사'], [], [], []]
[[], [], [], [], []]
[['감:명사', '감:동사'], [], [], [], [], ['는:보조사', '는:관형사형어미']]
[['가:동사'], [], [], ['기:명사형어미'], [], [], []]
[[], [], ['ㅁ:명사형어미'], [], [], [], [], []]



In [25]:
# step5: connectivity check
for idx_col in range(len(chars)-1, -1, -1):	# backward loop (len-1, len-2, ..., 0)
	for idx_row in range(idx_col+1, len(chars), 1):
		#print("row: ", idx_row, "col:", idx_col)	# index comparison for debugging
		#print len(table[idx_row][idx_col])	# length check for debugging
		idx_prime=len(chars)+idx_col-idx_row
		#print('compare '+str(idx_row)+','+str(idx_col)+' and '+str(idx_prime)+','+str(idx_prime))	# index comparison for debugging
		for idx_left_candidate in range(len(table[idx_row][idx_col])):
			for idx_right_candidate in range(len(table[idx_prime][idx_prime])):
				left_candidate = table[idx_row][idx_col][idx_left_candidate]
				left_tag = left_candidate.split('+')[-1].split(':')[1]

				right_candidate = table[idx_prime][idx_prime][idx_right_candidate]
				right_tag = right_candidate.split('+')[0].split(':')[1]
				if(left_tag+'\t'+right_tag in connectivity):
					table[idx_col][idx_col]+=[left_candidate+'+'+right_candidate]
					#print("position:", idx_col, idx_col)	# position check for debugging

print ("cyk table after connectivity checking:")
print_list(table)


cyk table after connectivity checking:
[['감기:명사+는:보조사', '감기:동사+는:관형사형어미', '감:동사+기:명사형어미+는:보조사']]
[[], []]
[[], [], []]
[['감기:명사', '감기:동사'], [], [], ['기:명사형어미+는:보조사']]
[[], [], [], [], []]
[['감:명사', '감:동사'], [], [], [], [], ['는:보조사', '는:관형사형어미']]
[['가:동사'], [], [], ['기:명사형어미'], [], [], []]
[[], [], ['ㅁ:명사형어미'], [], [], [], [], []]



In [26]:
# step6: print result
print ("result:")
print(table[0][0])

result:
['감기:명사+는:보조사', '감기:동사+는:관형사형어미', '감:동사+기:명사형어미+는:보조사']
